In [1]:
# IMPORT LIBRARY
import os
import time
import random
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input

In [2]:
# PARAMETER GLOBAL & SEED
FINAL_DATASET = "D:/KULIAH/SEMESTER 7/Skripsi/Dataset/Dataset_TrashNet_Final"

CLASSES = ["cardboard", "glass", "metal", "paper", "plastic", "trash"]

IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 1e-4
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [3]:
# DATA GENERATOR (RESCALE ONLY)
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_test_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_data = train_gen.flow_from_directory(
    os.path.join(FINAL_DATASET, "train"),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

val_data = val_test_gen.flow_from_directory(
    os.path.join(FINAL_DATASET, "val"),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

test_data = val_test_gen.flow_from_directory(
    os.path.join(FINAL_DATASET, "test"),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

Found 2001 images belonging to 6 classes.
Found 377 images belonging to 6 classes.
Found 383 images belonging to 6 classes.


In [4]:
# BASE MODEL: MobileNetV3 Small
base_model = MobileNetV3Small(
    weights="imagenet",
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

base_model.trainable = False

In [5]:
# Build model (APPLE TO APPLE)
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(64, activation="relu"),
    Dense(len(CLASSES), activation="softmax")
])

# COMPILE MODEL
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ MobileNetV3Small (Functional)        │ (None, 7, 7, 576)           │         939,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 576)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │             390 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 976,438 (3.72 MB)

 Trainable params: 37,318 (145.77 KB)

 Non-trainable params: 939,120 (3.58 MB)

In [6]:
# ESTIMASI UKURAN MODEL (TANPA SAVE MODEL)
total_params = model.count_params()

# default dtype MobileNet = float32 (4 byte)
BYTES_PER_PARAM = 4  

estimated_size_mb = (total_params * BYTES_PER_PARAM) / (1024 * 1024)

print("\nEstimated Model Size (In-Memory)")
print(f"Total parameter      : {total_params:,}")
print(f"ukuran model: {estimated_size_mb:.2f} MB (float32)")


Estimated Model Size (In-Memory)
Total parameter      : 976,438
ukuran model: 3.72 MB (float32)


In [6]:
# EARLY STOPPING
early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True,
    verbose=1
)

In [7]:
# TRAINING + TIME MEASUREMENT
start_time = time.time()

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    callbacks=[early_stopping],
    verbose=1
)

end_time = time.time()

total_training_time = end_time - start_time
epochs_ran = len(history.history["loss"])

print(f"\nTotal Training Time : {total_training_time:.2f} seconds")
print(f"Average / Epoch    : {total_training_time / epochs_ran:.2f} seconds")
print(f"Training stopped at epoch : {epochs_ran}")

Epoch 1/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 9s 96ms/step - accuracy: 0.3123 - loss: 1.7644 - val_accuracy: 0.5093 - val_loss: 1.4234
Epoch 2/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 78ms/step - accuracy: 0.5212 - loss: 1.3533 - val_accuracy: 0.5995 - val_loss: 1.1811
Epoch 3/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 81ms/step - accuracy: 0.6187 - loss: 1.1320 - val_accuracy: 0.6578 - val_loss: 1.0242
Epoch 4/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 82ms/step - accuracy: 0.6792 - loss: 0.9853 - val_accuracy: 0.6737 - val_loss: 0.9125
Epoch 5/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 80ms/step - accuracy: 0.7201 - loss: 0.8846 - val_accuracy: 0.6950 - val_loss: 0.8654
Epoch 6/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 80ms/step - accuracy: 0.7451 - loss: 0.8096 - val_accuracy: 0.7241 - val_loss: 0.7897
Epoch 7/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 80ms/step - accuracy: 0.7651 - loss: 0.7514 - val_accuracy: 0.7347 - val_loss: 0.7512
Epoch 8/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 80ms/step - accuracy: 0.7866 - loss: 0.7022 - val_accuracy: 0.7374 - v

In [8]:
# Waktu Inferensi Keseluruhan Test Set
start_time = time.time()

predictions = model.predict(test_data)

end_time = time.time()

inference_time_total = end_time - start_time
num_samples = test_data.samples

print(f"Total Inference Time (Test Set): {inference_time_total:.4f} seconds")
print(f"Average Inference Time per Image: {inference_time_total / num_samples:.6f} seconds")

12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 129ms/step
Total Inference Time (Test Set): 2.2277 seconds
Average Inference Time per Image: 0.005816 seconds


In [9]:
# Evaluasi Model (Test Set – Tetap Sama)
test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc:.4f}")

12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - accuracy: 0.7937 - loss: 0.6248 
Test Accuracy: 0.7937


In [10]:
# Ambil 1 batch
x_batch, _ = next(test_data)

# Warm-up (penting untuk CNN)
_ = model.predict(x_batch[:1])

start_time = time.time()
_ = model.predict(x_batch[:1])
end_time = time.time()

print(f"Inference Time (Single Image): {(end_time - start_time):.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 638ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Inference Time (Single Image): 0.062089 seconds


In [11]:
# CONFUSION MATRIX & REPORT
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

y_true = test_data.classes
y_pred = np.argmax(model.predict(test_data), axis=1)

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=CLASSES))

print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))

12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step

Classification Report:
              precision    recall  f1-score   support

   cardboard       0.91      0.84      0.87        61
       glass       0.78      0.75      0.77        76
       metal       0.80      0.82      0.81        62
       paper       0.86      0.82      0.84        90
     plastic       0.74      0.76      0.75        72
       trash       0.53      0.73      0.62        22

    accuracy                           0.79       383
   macro avg       0.77      0.79      0.78       383
weighted avg       0.80      0.79      0.80       383


Confusion Matrix:
[[51  1  1  6  0  2]
 [ 0 57  9  1  8  1]
 [ 1  3 51  2  2  3]
 [ 4  1  0 74  7  4]
 [ 0 11  2  0 55  4]
 [ 0  0  1  3  2 16]]
